In [73]:
import pandas as pd
import numpy as np

In [107]:
# import cost of living csv
coli_df = pd.read_csv("coli_composite.csv")
coli_df.head()

,Urban Area,State,COLI Composite
0,Anniston-CalhounCounty,AL,87.4
1,Auburn-Opelika,AL,92.3
2,Birmingham,AL,91.2
3,Decatur-Hartselle,AL,85.1
4,Dothan,AL,87.0


In [108]:
# import median rent per square feet by city (zillow)
mrsqft_df = pd.read_csv("median_rents_sqft.csv")
mrsqft_df.head()

,Region Name,State,2018-01
0,New York,NY,1.542
1,Los Angeles,CA,1.956
2,Chicago,IL,1.178
3,Dallas,TX,0.926
4,Philadelphia,PA,1.040


In [109]:
# import median property value per square feet by city (zillow)
mvsqft_df = pd.read_csv("median_value_sqft.csv")
mvsqft_df.head()

,Region Name,State,2018-01
0,New York,NY,255
1,Los Angeles,CA,423
2,Chicago,IL,142
3,Dallas,TX,118
4,Philadelphia,PA,140


In [110]:
# import hq2 city list
city_list = pd.read_csv("amazon_hq2_finalists.csv")
city_list.head()

,City/Region,State
0,Atlanta,GA
1,Austin,TX
2,Boston,MA
3,Chicago,IL
4,Columbus,OH


In [117]:
new_df = pd.merge(city_list, mvsqft_df,  how='left', left_on=['City/Region','State'], right_on = ['Region Name','State'])
del new_df['Region Name']
new_df.columns = ['Region', 'State', 'Median Value / Sqft']
new_df = pd.merge(new_df, mrsqft_df,  how='left', left_on=['Region','State'], right_on = ['Region Name','State'])
del new_df['Region Name']
new_df.columns = ['Region', 'State', 'Median Value / Sqft', 'Median Rent / Sqft']

,Region,State,Median Value / Sqft,Median Rent / Sqft
0,Atlanta,GA,103,0.782
1,Austin,TX,151,0.988
2,Boston,MA,271,1.514
3,Chicago,IL,142,1.178
4,Columbus,OH,114,0.892
5,Dallas,TX,118,0.926
6,Denver,CO,248,1.382
7,Indianapolis,IN,88,0.760
8,Los Angeles,CA,423,1.956
9,Miami,FL,176,1.382


In [118]:
merged_df = pd.merge(new_df, coli_df,  how='left', left_on=['Region','State'], right_on = ['Urban Area','State'])
merged_df

,Region,State,Median Value / Sqft,Median Rent / Sqft,Urban Area,COLI Composite
0,Atlanta,GA,103,0.782,Atlanta,98.8
1,Austin,TX,151,0.988,Austin,97.3
2,Boston,MA,271,1.514,Boston,147.9
3,Chicago,IL,142,1.178,Chicago,123.4
4,Columbus,OH,114,0.892,Columbus,89.6
5,Dallas,TX,118,0.926,Dallas,101.9
6,Denver,CO,248,1.382,Denver,111.8
7,Indianapolis,IN,88,0.760,Indianapolis,92.4
8,Los Angeles,CA,423,1.956,NaN,NaN
9,Miami,FL,176,1.382,NaN,NaN
